## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

In [269]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Merging Dataframes

1. 학생 정보 관련 파일:
   - studentInfo.csv: 학생의 인구 통계 정보, 등록 정보 등을 담고 있습니다.

2. 과정 정보 관련 파일:
   - courses.csv: 각 코스(모듈)에 대한 정보를 담고 있습니다.
   - assessments.csv: 코스 내의 평가 정보 (과제, 시험 등)를 담고 있습니다.

3. 학생 성적 관련 파일:
    - studentAssessment.csv: 학생들의 평가 점수를 담고 있습니다.
    - studentRegistration.csv: 학생의 코스 등록 정보를 담고 있습니다.
    
4. VLE 활동 관련 파일:
   - vle.csv: VLE 활동에 대한 설명 정보를 담고 있습니다.
   - studentVle.csv: 학생의 VLE(Virtual Learning Environment) 활동 데이터를 담고 있습니다.

In [270]:
assessments_df = pd.read_csv("./data/assessments.csv")
courses_df = pd.read_csv("./data/courses.csv")
studentAssessment_df = pd.read_csv("./data/studentAssessment.csv")

# vle_df = pd.read_csv("./data/vle.csv")
# studentVle_df = pd.read_csv("./data/studentVle.csv")

studentInfo_df = pd.read_csv("./data/studentInfo.csv")
studentRegistration_df = pd.read_csv("./data/studentRegistration.csv")

In [ ]:
print('코스 내의 과제, 시험 정보',assessments_df.columns)
print('코스 정보',courses_df.columns)
print('학생 평가 점수',studentAssessment_df.columns)
print('학생 정보',studentInfo_df.columns)
print('학생의 등록 정보',studentRegistration_df.columns)
# print('vle 활동 정보',vle_df.columns)
# print('학생의 vle 활동 정보',studentVle_df.columns)

In [272]:
# 1. 학생 평가 점수와 과제 정보 병합
merged_df = studentAssessment_df.merge(assessments_df, on='id_assessment', how='left')
# 2. 학생 정보 추가
merged_df = merged_df.merge(studentInfo_df, on=['id_student', 'code_module', 'code_presentation'], how='left')
# 3. 학생 등록 정보 추가
merged_df = merged_df.merge(studentRegistration_df, on=['id_student', 'code_module', 'code_presentation'], how='left')
# 4. 코스 정보 추가
merged_df = merged_df.merge(courses_df, on=['code_module', 'code_presentation'], how='left')

In [273]:
# 중복 행 제거
merged_df = merged_df.drop_duplicates()

In [274]:
merged_df.to_csv('merged_data.csv')

In [275]:
# merged_vle_df = pd.merge(vle_df, studentVle_df, 
#                            on=['id_site', 'code_module', 'code_presentation'],
#                            how='outer')

In [276]:
# 중복 행 제거
# merged_vle_df = merged_vle_df.drop_duplicates()

In [277]:
# merged_vle_df.to_csv('./data/merged_vle_data.csv')

---
# 데이터 전처리

### 1. 결측치 처리

### `date`

- 대부분 `assessment_type = "Exam"`에 해당
- 시험은 일반적으로 강의 종료일에 시행됨
→ `module_presentation_length`를 활용해 **강의 마지막 날짜로 채움**

In [278]:
merged_data=merged_df
merged_data['is_late'] = (merged_data['date_submitted'] > merged_data['date']).astype(int)

In [279]:
# # # 2. 컬럼 이름 정리 (공백 제거)
# # merged_data.columns = merged_data.columns.str.strip()
# # courses.columns = courses.columns.str.strip()

# # 3. 'Exam'이고 date가 NaN인 행 필터링
# is_exam_nan = (merged_data['assessment_type'] == 'Exam') & (merged_data['date'].isna())

# # 4. 강의별 강의일수 정보 가져오기 (module_presentation_length)
# module_length_map = merged_data.set_index(['code_module', 'code_presentation'])['module_presentation_length']

# # 5. 결측 date 값 채우기
# def fill_exam_date(row):
#     if row['assessment_type'] == 'Exam' and pd.isna(row['date']):
#         return module_length_map.get((row['code_module'], row['code_presentation']), row['date'])
#     return row['date']

# merged_data['date'] = merged_data.apply(fill_exam_date, axis=1)


### 🔹 결측치 제거

### `score`

- 총 173건 결측
    - `Withdrawn`(이탈자): 72개
    - `Pass`, `Fail`, `Distinction`(수료자): 101개
- 과제를 제출했는데도 점수가 없는 경우로 추정… 수가 적으므로 **삭제 처리**

### `date_registration`

- 수강 등록일이 없는 7건 → **삭제**

In [ ]:
# score가 NaN인 사람들 중에 이탈하지 않은 사람
score_nan_but_not_withdrawn = merged_data[merged_data['score'].isna() & (merged_data['final_result'] != 'Withdrawn')]

# score가 NaN인 사람들 중에 이탈한 사람
score_nan_and_withdrawn = merged_data[merged_data['score'].isna() & (merged_data['final_result'] == 'Withdrawn')]

# 결과 확인
# print(score_nan_but_not_withdrawn[['id_student', 'id_assessment', 'final_result']])
print(f"수료했는데도 score가 결측인 사람 수: {len(score_nan_but_not_withdrawn)}명")

# print(score_nan_and_withdrawn[['id_student', 'id_assessment', 'final_result']])
print(f"이탈했고 score가 결측인 사람 수: {len(score_nan_and_withdrawn)}명")


# Score 결측치 데이터(173개) + date_registration 결측치 데이터(7개) 데이터 제거

df_cleaned = merged_data.dropna(subset=['score','date_registration'])

df=df_cleaned


### `imd_band`

- 총 7697건 결측
- 전체 28,785명 중 약 **971명**의 `imd_band`(지역 기반 사회경제 수준 지표)가 없음

- imd_band에 데이터를 넣는코드

In [281]:
# 'imd_band' 컬럼의 NaN 값을 'Unknown'으로 채우기
# unknown 대신 다른 문자열로 채울 수 있음
# 숫자로도 넣을 수 있음.
df.loc[:, 'imd_band'] = df['imd_band'].fillna('Unknown')

- 'imd_band' 컬럼에서 결측치가 있는 행을 삭제

In [282]:
df = df.dropna(subset=['imd_band','date'])

In [ ]:
df.isnull().sum()

In [ ]:
print(df['imd_band'].isna().sum())  # NaN 개수 확인
print(df['imd_band'].unique())  # imd_band 컬럼의 고유값 확인

In [ ]:
display(df['imd_band'].unique())

imd_band_to_income = {
    '90-100%': 9,  
    '80-90%': 8,  
    '70-80%': 7, 
    '60-70%': 6, 
    '50-60%': 5, 
    '40-50%': 4, 
    '30-40%': 3,  
    '20-30%': 2,    
    '10-20': 1, 
    '0-10%': 0,    
    'Unknown': 0     
}

df['imd_band'] = df['imd_band'].map(imd_band_to_income)

### `date_unregistration`

- 결측치 160,857건
- 해당 컬럼은 **"언제 수강을 중도 이탈했는가"**를 의미
→ **결측 = 중도 이탈하지 않은 수료자**
→ 즉, 의미 있는 결측이므로 **삭제하지 않음** → 인코딩 과정에서 결측치를 처리 한다!!!

---

- date_unregistration 이상치 제거
- Fail값이 나올 수 없기 때문에 이상치 처리해야함!!!!!!!!!!!!!!!!!!!!
- 조건 -> final_result 값 안에 Fail 일 때 date_unregistration이 0보다 크거나 같을 때

### `final_result 이상치 처리 코드`

In [ ]:
df[(df['final_result'] == 'Fail') & (df['date_unregistration'] >= 0)]

In [287]:
df = df.drop(df[(df['final_result'] == 'Fail') & (df['date_unregistration'] >= 0)].index)

In [288]:
merged_data=df

### 2. 이상치 처리 및 시각화
   - 수치형 변수의 이상치 확인 및 처리(예: score, studied_credits의 극단값 혹은 0 값 등)

### 3. 데이터 타입 변환
   - 날짜 관련 변수를 datetime 형식으로 변환
   - 범주형 변수를 category 타입으로 변환

### 4. 특성 엔지니어링

#### 학생의 성적 관련 특성
  - 각 학생의 평균 점수, 최고 점수, 최저 점수, 점수의 표준편차
    - 각 학생당 코스별 성적 편차 필요할까?
  - 점수 추세 (상승 또는 하락)
  - 과제 난이도에 따른 가중 점수 -> 난이도 기준을 뭘로 잡아야하나?
 
##### my_average_score, my_max_score, my_min_score, my_score_std, my_score_trend, assesment_weight, weighted_score

- my_avg_score : 개인 학업 성취도 수준 파악
- my_max/min_score/my_score_std : 특정 과목 강점/약점 식별 및 극단적 편차 분석
- my_score_trend	: 학습 효과성 평가 (지속적 상승=효율적 학습법, 하락=개입 필요)
- weighted_score :	난이도 대비 성취도 → "B과제는 고난이도지만 고가중점수 → Distinct 학생" 

In [289]:
# 학생의 평균, 최고, 최저 점수 및 표준편차 계산
student_scores = merged_data.groupby('id_student')['score'].agg(['mean', 'max', 'min', 'std']).reset_index()
student_scores.columns = ['id_student', 'my_average_score', 'my_max_score', 'my_min_score', 'my_score_std']

# # 학생의 코스별 성적 표준편차 계산
# score_std_by_module = merged_data.groupby(['id_student', 'code_module'])['score'].std().reset_index()
# score_std_by_module = score_std_by_module.pivot(index='id_student', columns='code_module', values='score').reset_index()
# score_std_by_module.columns = ['id_student'] + [f'score_std_{col}' for col in score_std_by_module.columns if col != 'id_student']

# 점수 추세 분석
merged_data_sorted = merged_data.sort_values(by=['id_student', 'date_submitted'])
merged_data_sorted['score_diff'] = merged_data_sorted.groupby('id_student')['score'].diff()

def determine_trend(diff):
    if diff > 0:
        return 'increasing'
    else:
        return 'decreasing'
    
merged_data_sorted['score_trend'] = merged_data_sorted['score_diff'].apply(determine_trend)
student_trends = merged_data_sorted.groupby('id_student')['score_trend'].apply(lambda x: x.value_counts().idxmax()).reset_index()
student_trends.columns = ['id_student', 'my_score_trend']

# 모든 특성을 하나의 데이터프레임으로 병합
student_features = pd.merge(student_scores, student_trends, on='id_student')

# 원본 데이터와 새로운 특성을 병합
merged_data = pd.merge(merged_data, student_features, on='id_student')

# 과제 난이도 가중치를 적용한 점수 (평균 기반)

# weight 계산 (평균 점수가 낮을수록 가중치가 높아짐)
avg = merged_data.groupby('id_assessment')['score'].mean()
max_avg = avg.max()
weight = (max_avg - avg) / (max_avg - avg).sum()

# weight를 데이터프레임으로 변환
weight_df = weight.reset_index()
weight_df.columns = ['id_assessment', 'assessment_weight']
merged_data = pd.merge(merged_data, weight_df, on='id_assessment', how='left')

# 가중 점수 계산
merged_data['weighted_score'] = merged_data['score'] * merged_data['assessment_weight']

#### 코스 관련 특성
  - 코스별 평균 점수, 최고 점수, 최저 점수, 점수의 표준편차 -> 최저 점수 다 0임, 최고점이 필요한가?, 중간값이 필요한가?
  - 코스별 과제 개수

##### 'course_avg_score', 'course_std_score', 'assignment_count'

- course_avg_score, course_std_score :	평가 방식 적정성 → 편차↑=과도한 변별력 / 강좌 난이도 레벨링
- assignment_count :	과제량-성적 상관관계 분석 → 최적 과제량 도출

In [290]:
# 과목별 통계 계산
subject_stats = merged_data.groupby(['code_presentation', 'code_module']).agg({
    'score': ['mean', 'max', 'std']
})
subject_stats.columns = ['course_avg_score', 'course_max_score', 'course_std_score']

# 과목 통계 데이터 병합
merged_data = merged_data.merge(
    subject_stats.reset_index(),
    on=['code_presentation', 'code_module'],
    how='left'
)
merged_data['assignment_count'] = merged_data.groupby(['code_presentation', 'code_module'])['id_assessment'].transform('nunique')

# # 과목별 난이도 가중치를 적용한 점수 (표준편차 기반)
# difficulty = subject_stats['course_std_score']
# weights = (difficulty / difficulty.sum()) * 10
# merged_data = merged_data.merge(
#     weights.rename('course_weight'), 
#     left_on='code_module', 
#     right_index=True
# )
# merged_data['weighted_score'] = merged_data['score'] * merged_data['course_weight']

In [ ]:
merged_data[merged_data['code_module']=='AAA']

#### 행동 패턴 특성
- 전체 학생 기준
  - 코스별 과제 제출률 => 제출률 100프로
  - 코스별 지각 제출 비율
  - 과제별 제출 시간의 평균, 중앙값, 표준편차

- 한명 학생 기준
  - 과제 제출률 => 제출률 100프로
  - 지각 제출 비율
  - 이탈 학생의 이탈 단계 할당

##### 'course_late_rate', 'my_late_rate', 'days_early_submission_avg', 'days_early_submissione_max', 'days_early_submission_min','days_early_submission_std','days_early_submission', 'is_late', 'unregistration_stage'

- my_late_rate	: 습관적 지각 여부 → 학습태도 평가 지표
- days_early_submission_*	: 과제의 난의도 판별 → 평균 7일 이상 = 쉬운 과제인가, 평균 0일 이하 = 어려운 과제인가, 표준편차가 크다 = 학생의 역량
- course_late_rate : 코스의 난의도 판별 -> 필요없을까?
- unregistration_stage : 첫 번째 과제 이후 이탈-> 수업의 어려움보다는 외부적인 이유일 가능성이 높음 / 중간쯤 과제 제출 후 이탈->학업에서의 고충이나 과제의 난이도 등이 영향이 있을 수 있음

In [292]:
# 모든 학생 기준
# 1. 코스별 평균 제출률 계산
# 1-1. 학생별 코스 내 제출률 계산
# student_course_submission = merged_data.groupby(['code_module', 'id_student']).agg(
#     student_submissions=('id_assessment', 'count'),
#     course_assessments=('id_assessment', 'nunique')
# ).astype({'student_submissions': int, 'course_assessments': int}).reset_index()

# # 1-2. 학생별 제출률 계산
# student_course_submission['student_submission_rate'] = (
#     student_course_submission['student_submissions']
#     / student_course_submission['course_assessments']
# )
# course_stats = student_course_submission.groupby('code_module')['student_submission_rate'].mean().reset_index()
# course_stats.columns = ['code_module', 'course_avg_submission_rate']
# student_course_submission[student_course_submission['student_submission_rate'] != 1]

# 2. 코스별 지각 제출 비율
# 3. 과제별 제출시간 평균, 중간, 표준편차

# 개인기준
# # 1. 학생별 과제 제출률
# student_stats = merged_data.groupby('id_student').agg(
#     student_submissions=('id_assessment', 'count'),
#     total_assignments=('id_assessment', 'nunique')
# ).reset_index()
# student_stats['student_submission_rate'] = student_stats['student_submissions'] / student_stats['total_assignments']
# student_stats[student_stats['student_submission_rate'] != 1]

# 2. 학생별 지각 제출 비율

In [293]:
# 코스별 지각 제출 비율
# merged_data['is_late'] = (merged_data['date_submitted'] > merged_data['date']).astype(int)
merged_data['course_late_rate'] = merged_data.groupby(['code_presentation', 'code_module'])['is_late'].transform('mean')

# 과제별 데드라인 전 제출 시간 평균, 중간, 표준편차
merged_data['days_early_submission'] = merged_data['date'] - merged_data['date_submitted']  # 컬럼명 변경 권장
assessment_stats = merged_data.groupby('id_assessment')['days_early_submission'].agg([
    ('days_early_submission_avg', 'mean'),
    ('days_early_submission_max', 'max'),
    ('days_early_submission_min', 'min'),
    ('days_early_submission_std', 'std')
]).reset_index()
merged_data = merged_data.merge(assessment_stats, on='id_assessment')

# 학생별 지각 제출 비율
merged_data['my_late_rate'] = merged_data.groupby('id_student')['is_late'].transform('mean')

In [294]:
dropped_out_students = merged_data[merged_data['date_unregistration'].notnull()]

# 이탈한 학생이 제출한 과제 수 계산 (학생, 과목별로 그룹화)
submitted_assignments = dropped_out_students.groupby(['id_student', 'code_presentation', 'code_module'])['id_assessment'].count().reset_index()
submitted_assignments.rename(columns={'id_assessment': 'submitted_count'}, inplace=True)
merged_data = merged_data.merge(submitted_assignments, on=['id_student', 'code_presentation', 'code_module'], how='left')

# 이탈 학생의 과제 제출률 계산 (이탈 학생의 과목당 제출 과제 수 / 과목당 전체 과제 수)
merged_data['submitted_proportion'] = merged_data['submitted_count'] / merged_data['assignment_count']
merged_data['submitted_proportion'] = merged_data['submitted_proportion'].fillna(0)

# 이탈 학생의 이탈 단계 할당
def assign_stage_based_on_proportion(row):
    if pd.isna(row['date_unregistration']):
        return 0
    elif row['submitted_proportion'] < 0.33:  # Less than 33% of total assignments
        return 1
    elif row['submitted_proportion'] < 0.66:  # Between 33% and 66%
        return 2
    else:                                     # More than 66%
        return 3

merged_data['unregistration_stage'] = merged_data.apply(assign_stage_based_on_proportion, axis=1)

In [ ]:
merged_data = merged_data.drop(['submitted_count','assignment_count','date_unregistration'], axis=1)
merged_data.isnull().sum()

In [ ]:
merged_data.loc[merged_data['unregistration_stage'].notnull(), 
                ['id_student','code_presentation', 'code_module','unregistration_stage']]


#### 아래부터는 EDA 아이디어가 정해진 후 진행

### 5. 불필요한 특성 제거
   - 분석에 불필요하거나 중복되는 정보를 가진 열 제거

### 6. 범주형 변수 인코딩
   - 필요에 따라 원-핫 인코딩 또는 라벨 인코딩 적용

In [297]:
df=merged_data

In [298]:

df.loc[:, 'final_result'] = df['final_result'].apply(lambda x: 1 if x == 'Withdrawn' else 0)

# df.loc[:, 'date_unregistration'] = df['date_unregistration'].apply(lambda x: 1 if pd.notna(x) else 0)

# -> 조건을 주지 않고 원 핫 인코딩을 실행함
encode_cols = ['code_module', 'code_presentation', 'assessment_type']
df_encoded = pd.get_dummies(df[encode_cols], drop_first=False)

# 'age_band' 값을 매핑하여 숫자로 변환
df['age_band'] = df['age_band'].astype(str)
age_band_map = {'0-35': 0, '35-55': 1, '55<=': 2}
df['age_band'] = df['age_band'].map(age_band_map)

# 원핫인코딩 제외한 나머지 컬럼 가져오기
df_other = df.drop(columns=encode_cols)

# 변환된 데이터프레임 합치기
df = pd.concat([df_other, df_encoded], axis=1)

# 결과 확인
df['final_result'].value_counts()
df['final_result']=df['final_result'].astype(int)


In [ ]:
df = df[merged_data['is_banked']==0]
df = df.drop('is_banked', axis=1)


education_to_income = {
    'Post Graduate Qualification': 6926,  
    'HE Qualification': 4699,      
    'A Level or Equivalent': 3630,     
    'Lower Than A Level': 3576,        
    'No Formal quals': 3000     
}

df['highest_education'] = df['highest_education'].map(education_to_income)
df.info()

### 7. 스케일링(-> 모델링 시점에 할 지 안할지 정해도 될듯)
   - 수치형 변수에 대해 표준화 또는 정규화 적용

---
# **Exploratory Data Analysis**

## **특성간의 상관관계 확인**
- 특성간의 상관관계 확인 후 상관계수가 높은 변수끼리 그룹화하여 그룹별 성과와의 상관관계 분석

In [ ]:
df.info()
df.columns

In [301]:
df.to_csv('merged_data_finall.csv')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 상관 행렬 계산
matrix = df.drop(columns=['my_min_score','my_max_score', 'id_assessment', 'id_student','score','is_late','days_early_submission_max','days_early_submission_min','days_early_submission_std','days_early_submission_avg'])
corr_matrix = matrix.corr(numeric_only=True)
corr_matrix
# # 상삼각 행렬 마스킹
# upper_mask = np.triu_indices_from(corr_matrix, k=1)
# high_corr_values = corr_matrix.values[upper_mask]
# average_corr = high_corr_values.mean()

# # 상/하위 필터링 (상삼각만 적용)
# high_corr = corr_matrix.where(
#     (corr_matrix > average_corr) & 
#     np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
# )
# low_corr = corr_matrix.where(
#     (corr_matrix < average_corr) & 
#     np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
# )

# # 상위 20개 추출
# top_20 = high_corr.unstack().dropna().sort_values(ascending=False).head(20)
# down_20 = low_corr.unstack().dropna().sort_values().head(20)

# # 변수 목록 생성 (집합 → 리스트 변환)
# def get_unique_vars(pairs):
#     return list({var for pair in pairs.index for var in pair})

# vars_to_plot = get_unique_vars(top_20)
# vars_to_plot2 = get_unique_vars(down_20)

# # 히트맵 생성
# def plot_heatmap(data, title):
#     plt.figure(figsize=(12,10))
#     sns.heatmap(data, annot=True, cmap='coolwarm', fmt=".2f", 
#                 center=0, vmin=-1, vmax=1)
#     plt.title(title)
#     plt.show()

# plot_heatmap(corr_matrix.loc[vars_to_plot, vars_to_plot], 
#             "Top 20 Highest Correlations")
# plot_heatmap(corr_matrix.loc[vars_to_plot2, vars_to_plot2], 
#             "Bottom 20 Lowest Correlations")

# 상위 20개 컬럼만 선택
top_20_columns = corr_matrix.columns[:30]
top_20_corr_matrix = corr_matrix.loc[top_20_columns, top_20_columns]

# 히트맵 생성
plt.figure(figsize=(12, 10))
sns.heatmap(top_20_corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', cbar=True)
plt.title('Top 20 Columns Correlation Matrix Heatmap')
plt.show()

In [303]:
# from itertools import combinations
# merged_data=matrix
# numeric_df = merged_data.select_dtypes(include=['int64', 'float64'])

# column_pairs = list(combinations(numeric_df.columns, 2))

# for x_col, y_col in column_pairs:
#     plt.figure(figsize=(6, 4))
#     sns.scatterplot(data=numeric_df, x=x_col, y=y_col, alpha=0.5)
#     plt.title(f"Scatter Plot: {x_col} vs {y_col}")
#     plt.tight_layout()
#     plt.show()

In [304]:
# numeric_df = merged_data.select_dtypes(include=['int64', 'float64'])

# for col in numeric_df.columns:
#     if col != 'date_unregistration':  # 'date_unregistration' ??
#         plt.figure(figsize=(6, 4))
#         sns.scatterplot(data=numeric_df, x='date_unregistration', y=col, alpha=0.5)
#         plt.title(f"Scatter Plot: date_unregistration vs {col}")
#         plt.tight_layout()
#         plt.show()

### 여기부터는 위의 상관관계 확인후 분석 들어가야함

## (예시)**온라인 학습 활동 분석**
- sum_click(총 클릭 수)와 final_result(최종 성적) 간 상관관계 분석
- 학습 콘텐츠 유형(activity_type)별 평균 클릭 수(sum_click) 비교
- week_from, week_to를 활용한 주차별 학습 패턴 분석 (lineplot)

## (예시)**학생의 연령, 교육 수준, 장애 여부 등이 학습 성과와 관련 있는지 분석**
- age_band, highest_education, disability 등과 final_result 간 분포 비교 (countplot)
- num_of_prev_attempts(재수강 횟수)와 final_result 간 상관관계 분석
- date_unregistration 값이 있는 경우, 조기 이탈 학생과 학습 활동 비교

---

### randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_clf = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=0)

merged_data=df
# 학습
X_train = merged_data.drop('final_result',axis=1)
y_train = merged_data['final_result']

rf_clf.fit(X_train, y_train)

y_pred_train = rf_clf.predict(X_train)
acc_score_train = accuracy_score(y_train, y_pred_train)
print('학습 점수:', acc_score_train)

# y_pred_test = rf_clf.predict(X_test)
# acc_score_test = accuracy_score(y_test, y_pred_test)
# print('테스트 평가 점수:', acc_score_test)

In [ ]:
feat_imptc_ser = pd.Series(rf_clf.feature_importances_, index=data.feature_names).sort_values(ascending=False)
feat_imptc_ser

# 특성 중요도 시각화
plt.figure(figsize=(8, 6))
sns.barplot(
    x=feat_imptc_ser,
    y=feat_imptc_ser.index,
    hue=feat_imptc_ser.index
)